# Missing Values

In this section, we'll be learning different missing value imputation techniques by analyzing simulated customer lifetime value data. Customer Lifetime Value is the total monetary worth the customer has to the business, over the course of its business-customer relationship. In this dataset, to keep things simple, we'll be using purchases as a proxy for CLV. 

We'll be covering how to:

- Check the number of null values

- Dropping Null Values

- Mean/Median/Mode Imputation

- Multiple Imputation using Regression

- Imputation using Nearest Neighbors

Let's get started!

## Import Libraries

First, we'll need to import the relevant libraries. We'll be using the standard `pandas`, `numpy` libraries for data manipulation. We'll then be using `sklearn` for the more advanced imputation techniques. We will use `scipy` for the `mode` imputation later on:

In [15]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

import warnings  
warnings.filterwarnings("ignore")

## Load Data

Next, we'll load our customer lifetime value dataset. You'll see in our dataset, we have about 6 columns. The `purchases` column is the column we care about in our customer lifetime value problem. 

In [4]:
import random

df = pd.read_csv("clv_data.csv")

df['lifetime_value'] = df['purchases'] * 20

df.head()

,Unnamed: 0,id,age,gender,income,days_on_platform,city,purchases,lifetime_value
0,0,0,NaN,Male,126895,14.0,San Francisco,0,0
1,1,1,NaN,Male,161474,14.0,Tokyo,0,0
2,2,2,24.0,Male,104723,34.0,London,1,20
3,3,3,29.0,Male,43791,28.0,London,2,40
4,4,4,18.0,Female,132181,26.0,London,2,40


In [5]:
random.randint(5,50)

42

## Checking Null Values

The first step in any data analysis or ML model is to check null values. We can check the number of nulls in a single line:

In [6]:
df.isnull().sum()

Unnamed: 0             0
id                     0
age                 2446
gender                 0
income                 0
days_on_platform     141
city                   0
purchases              0
lifetime_value         0
dtype: int64

In [7]:
null_values = pd.DataFrame(df.isnull().sum())
null_values['null_pct'] = null_values[0]/len(df)
null_values

,0,null_pct
Unnamed: 0,0,0.0000
id,0,0.0000
age,2446,0.4892
gender,0,0.0000
income,0,0.0000
days_on_platform,141,0.0282
city,0,0.0000
purchases,0,0.0000
lifetime_value,0,0.0000


However, if you want to see the percentages, we wrote a function you can use:

In [8]:
def nulls_summary_table(df):
    """
    Returns a summary table showing null value counts and percentage
    
    Parameters:
    df (DataFrame): Dataframe to check
    
    Returns:
    null_values (DataFrame)
    """
    null_values = pd.DataFrame(df.isnull().sum())
    null_values[1] = null_values[0]/len(df)
    null_values.columns = ['null_count','null_pct']
    return null_values

nulls_summary_table(df)

,null_count,null_pct
Unnamed: 0,0,0.0000
id,0,0.0000
age,2446,0.4892
gender,0,0.0000
income,0,0.0000
days_on_platform,141,0.0282
city,0,0.0000
purchases,0,0.0000
lifetime_value,0,0.0000


### Dropping Null Values

Dropping nulls is the quickest and easiest method to dropping nulls. We will use the internal pandas method `dropna` which will simply drop all rows that contain nulls:

In [9]:
drop_df = df.copy()

drop_df = drop_df.dropna()

In [16]:
X_d = drop_df[['age','days_on_platform','income']]
y_d = drop_df['lifetime_value']


X_train_d = X_d[:4000]
y_train_d = y_d[:4000]

X_test_d = X_d[1000:]
y_test_d = y_d[1000:]

### Mean/Median/Mode Imputation

The next is mean/median/mode imputation. We can use the native numpy functions for the mean and median. We can use scipy for the mode. Then, pandas as a native `fillna` method we can use to impute the nulls with the mean/median/mode:

In [17]:
m_df = df.copy()

X_m = m_df[['age','days_on_platform','income']]
y_m = m_df['lifetime_value']


X_train_m = X_m[:4000]
y_train_m = y_m[:4000]

X_test_m = X_m[1000:]
y_test_m = y_m[1000:]

In [18]:
## Mean
X_train_m.loc[:,'age'] = X_train_m['age'].fillna(np.mean(X_train_m['age']))
X_test_m.loc[:,'age'] = X_test_m['age'].fillna(np.mean(X_train_m['age'])) ## Cannot use training dataset to impute


X_train_m.loc[:,'days_on_platform'] = X_train_m['days_on_platform'].fillna(
                                        np.mean(X_train_m['days_on_platform']))
X_test_m.loc[:,'days_on_platform'] = X_test_m['days_on_platform'].fillna(np.mean(
                                    X_train_m['days_on_platform'])) ## Cannot use training dataset to impute

In [13]:
## Median
m_df.loc[:,'age'] = df['age'].fillna(np.median(m_df['age']))

In [31]:
## Mode
m_df.loc[:,'age'] = m_df['age'].fillna(stats.mode(m_df['age'])[0][0])

### Multiple Imputation Using Regression

Now that we've covered the simpler imputation techniques, we'll cover a more complicated imputation technique: Multiple Imputation. Multiple Imputation requires you to have some knowledge of ML modeling because we are using an ML model to impute the missing values. We won't go over every argument, but we'll go over the key ones. 

Multiple imputation has a few different estimators, using the `estimator` argument:

- `BayesianRidge`: Regularized Linear Regression

- `RandomForestRegressor`: Random Forest Model. Mimics missForest in the R language.

We'll go over how these estimators work in the next course: ML Algorithms. 

The `missing_values` argument is a placeholder for the data type of the missing values you want to impute. 

It's important to use `add_indicatorbool` as it'll create a placeholder indicating that you've imputed a missing value. This is important, because there could be patterns behind how a value is missing. Adding an indicator would allow you to keep track of where you made an imputation. Plus, it could also add signal into your model. 

`max_iter`: The number of iteration rounds.

In [20]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

## Target - Purchases in the first six months

r_df = df.copy()

X_r = r_df[['age','days_on_platform','income']]
y_r = r_df['lifetime_value']


X_train_r = X_r[:4000]
y_train_r = y_r[:4000]

X_test_r = X_r[1000:]
y_test_r = y_r[1000:]


Imp = IterativeImputer(max_iter=10, random_state = 0)
Imp.fit(X_train_r)

X_train_r = Imp.transform(X_train_r)
X_test_r = Imp.transform(X_test_r)

X_train_r = pd.DataFrame(X_train_r)
X_train_r.columns = X_train_r.columns

X_test_r = pd.DataFrame(X_test_r)
X_test_r.columns = X_test_r.columns

r_df = pd.concat([X_train_r,X_test_r],axis = 0)

In [25]:
X_train_r = pd.DataFrame(X_train_r)
X_train_r.columns = X_train_r.columns
X_train_r

,0,1,2
0,30.711715,14.0,126895.0
1,31.013485,14.0,161474.0
2,24.000000,34.0,104723.0
3,29.000000,28.0,43791.0
4,18.000000,26.0,132181.0
...,...,...,...
3995,31.635369,10.0,232795.0
3996,35.000000,1.0,46961.0
3997,46.000000,52.0,51432.0
3998,14.000000,37.0,49409.0


### Nearest Neighbor Imputation

On top of using linear regression or random forest regression to impute values, you can also use nearest neighbors imputation. Nearest neighbor imputation essentially uses a K-Nearest Neighbors algorithm to find the most similar data points, to impute the null values. 

**Important Parameters**

`missing_values`: This is the type of null value you want to impute. Typically, this is NaN, but it could be float or whichever you decide. 

`n_neighbors`: The number of neighbors to use for imputation. You can add more or less. Fewer neighbors can lead to overfitting. Larger numbers will lose some precision. 

`weights`: Pick how you want to weight all points in each neighborhood. There are two typical ways: `'uniform'` or `'distance'`. Uniform is equal weighting. Distance is weighted by the distance from the point. 

- `callable` : a user-defined function which accepts an array of distances, and returns an array of the same shape containing the weights.

`metric`: The distance metric used to search for neighbors. The default is euclidean.

`add_indicator`: This will add a dummy feature 0 or 1 if the value was imputed, similar to `add_indicatorbool`. 

In [21]:
imputer = KNNImputer(n_neighbors=5, weights="uniform")
imputer.fit(X_train_r)
X_train_k = imputer.transform(X_train_r)
X_test_k = imputer.transform(X_test_r)

y_train_k = y_train_r.copy()
y_test_k = y_test_r.copy()

## Comparison

In [26]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Drop Null Model
clf_n = RandomForestRegressor(random_state=0)
clf_n.fit(X_train_d, y_train_d)
pred_dropna = clf_n.predict(X_test_d)

# Mean Imputation Model
clf_m = RandomForestRegressor(random_state=0)
clf_m.fit(X_train_m, y_train_m)
pred_m = clf_m.predict(X_test_m)

# Regression Imputation
clf_r = RandomForestRegressor(random_state=0)
clf_r.fit(X_train_r, y_train_r)
pred_r = clf_r.predict(X_test_r)

#Nearest Neighbor Imputation
clf_n = RandomForestRegressor(random_state=0)
clf_n.fit(X_train_k, y_train_k)
pred_k = clf_n.predict(X_test_k)


print('Drop Null MAE Score: %.3f' % mean_absolute_error(y_test_d,pred_dropna))
print('Mean Impute MAE Score: %.3f' % mean_absolute_error(y_test_m,pred_m))
print('Regression MAE Score: %.3f '% mean_absolute_error(y_test_r,pred_r))
print('Nearest Neighbor MAE Score: %.3f'% mean_absolute_error(y_test_k,pred_k))

Drop Null MAE Score: 7.636
Mean Impute MAE Score: 10.828
Regression MAE Score: 10.795 
Nearest Neighbor MAE Score: 10.795


## Summary

In this section, you learned about a variety of missing value imputation techniques, ranging from simple to more complex. We went over:

- Dropping Nulls

- Mean/Median/Mode Imputation

- Regression Imputation (linear and forests)

- Nearest Neighbor Imputation